# 배터리 데이터 처리 시스템

배터리 충방전 데이터를 자동으로 분류하고 분석하는 범용 시스템입니다.

## 기능
- 자동 사이클러 분류 (Toyo/PNE)
- 메타 정보 추출
- 원시 데이터 로딩
- 시각화
- 결과 저장

## 1. Setup: 필수 라이브러리 임포트

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 로딩 완료")

## 2. 유틸리티 함수 정의

In [ ]:
def check_cycler(raw_file_path):
    """
    충방전기 구분 (패턴 폴더 유무로 구분)
    
    Parameters:
        raw_file_path (str): 분석할 데이터 경로
    
    Returns:
        str: 'PNE' 또는 'Toyo'
    """
    # 충방전기 데이터 폴더에 패턴 폴더 유무로 PNE와 Toyo 구분
    has_pattern = os.path.isdir(os.path.join(raw_file_path, "Pattern"))
    return "PNE" if has_pattern else "Toyo"


def name_capacity(data_file_path):
    """
    filepath 이름에서 용량을 추출하는 함수
    
    Parameters:
        data_file_path (str): 데이터 경로
    
    Returns:
        float or None: 추출된 용량 (mAh), 없으면 None
    """
    # 원시 문자열을 사용하여 특수 문자를 공백으로 대체
    raw_file_path = re.sub(r'[._@\$\(\)]', ' ', data_file_path)
    # 정규 표현식을 사용하여 "mAh"로 끝나는 용량 값을 찾습니다. (소수점 포함)
    match = re.search(r'(\d+([\-.]\d+)?)mAh', raw_file_path)
    if match:
        # 소수점 용량을 위해 -를 .으로 변환
        min_cap = match.group(1).replace('-', '.')
        return float(min_cap)
    return None


def get_directory_info(path):
    """
    디렉토리 메타 정보 추출
    
    Parameters:
        path (str): 분석할 디렉토리 경로
    
    Returns:
        dict: 폴더명, 서브폴더 개수, 파일 개수, Pattern 폴더 유무, 경로 존재 여부
    """
    info = {
        'path': path,
        'folder_name': os.path.basename(path),
        'exists': os.path.exists(path),
        'has_pattern': False,
        'num_subfolders': 0,
        'num_files': 0,
        'cycler_type': 'Unknown',
        'capacity_mAh': None
    }
    
    if info['exists']:
        # Pattern 폴더 확인
        info['has_pattern'] = os.path.isdir(os.path.join(path, "Pattern"))
        
        # 사이클러 타입 결정
        info['cycler_type'] = check_cycler(path)
        
        # 서브폴더 및 파일 개수
        try:
            items = os.listdir(path)
            for item in items:
                item_path = os.path.join(path, item)
                if os.path.isdir(item_path):
                    info['num_subfolders'] += 1
                else:
                    info['num_files'] += 1
        except PermissionError:
            pass
        
        # 용량 추출
        info['capacity_mAh'] = name_capacity(path)
    
    return info


print("✅ 유틸리티 함수 정의 완료")

## 3. 경로 입력

**여기에 분석할 배터리 데이터 경로를 입력하세요**

In [ ]:
# 분석할 경로 리스트 (사용자가 수정)
paths = [
    r"C:\Users\Ryu\Python_project\data\dataprocess\Rawdata\A1_MP1_4500mAh_T23_1",
    # 추가 경로를 여기에 입력하세요
    # r"D:\Battery_Data\Path2",
    # r"D:\Battery_Data\Path3",
]

print(f"분석 대상 경로 개수: {len(paths)}")
for i, path in enumerate(paths, 1):
    print(f"  {i}. {path}")

## 4. 자동 분류: 데이터 스캔 및 사이클러 타입 식별

In [ ]:
# 데이터 수집
results = []

for path in paths:
    print(f"분석 중: {path}")
    info = get_directory_info(path)
    results.append(info)

# DataFrame 생성
df_results = pd.DataFrame(results)

print(f"\n✅ 총 {len(df_results)}개 경로 분석 완료")

# 사이클러 타입별 통계
print("\n=== 사이클러 타입별 통계 ===")
print(df_results['cycler_type'].value_counts())

# 상세 결과 테이블
print("\n=== 상세 분류 결과 ===")
display_cols = ['folder_name', 'cycler_type', 'capacity_mAh', 'num_subfolders', 'num_files', 'has_pattern', 'exists']
print(df_results[display_cols].to_string(index=False))

## 5. 데이터 로딩 함수 정의

### PNE 및 Toyo 사이클러의 원시 데이터를 로드하는 함수

In [ ]:
# ========== PNE 데이터 로딩 함수 ==========

def load_pne_cycle_data(path):
    """
    PNE 사이클 데이터 로딩 (SaveEndData.csv)
    
    Parameters:
        path (str): PNE 데이터 경로
    
    Returns:
        pd.DataFrame or None: 사이클 데이터 DataFrame
    """
    restore_path = os.path.join(path, "Restore")
    
    if not os.path.isdir(restore_path):
        print(f"⚠️  Restore 폴더가 없습니다: {path}")
        return None
    
    # SaveEndData.csv 파일 찾기
    csv_files = [f for f in os.listdir(restore_path) if f.endswith('.csv')]
    end_data_file = None
    
    for file in csv_files:
        if 'SaveEndData' in file:
            end_data_file = file
            break
    
    if not end_data_file:
        print(f"⚠️  SaveEndData.csv 파일을 찾을 수 없습니다: {restore_path}")
        return None
    
    try:
        file_path = os.path.join(restore_path, end_data_file)
        if os.stat(file_path).st_size == 0:
            print(f"⚠️  빈 파일입니다: {end_data_file}")
            return None
        
        # CSV 로드 (컬럼 헤더 없음)
        df = pd.read_csv(file_path, sep=',', skiprows=0, engine='c', 
                        header=None, encoding='cp949', on_bad_lines='skip')
        
        # 필요한 컬럼만 선택: 27(사이클), 2(조건), 11(방전용량), 9(전류), 24(온도), 6(종료상태), 8(전압)
        df = df[[27, 2, 11, 9, 24, 6, 8]]
        df.columns = ['Cycle', 'Condition', 'DchgCap_mAh', 'Current_mA', 'Temp_C', 'EndState', 'Voltage_mV']
        
        # 온도 및 전압 단위 변환
        df['Temp_C'] = df['Temp_C'] / 1000
        df['Voltage_mV'] = df['Voltage_mV'] / 1000
        df['Current_mA'] = df['Current_mA'] / 1000
        
        print(f"✅ PNE 사이클 데이터 로드 완료: {len(df)} rows")
        return df
        
    except Exception as e:
        print(f"❌ PNE 사이클 데이터 로딩 실패: {e}")
        return None


def load_pne_profile_data(path, max_files=10):
    """
    PNE 프로파일 데이터 로딩 (SaveData*.csv)
    
    Parameters:
        path (str): PNE 데이터 경로
        max_files (int): 로드할 최대 파일 개수
    
    Returns:
        pd.DataFrame or None: 프로파일 데이터 DataFrame
    """
    restore_path = os.path.join(path, "Restore")
    
    if not os.path.isdir(restore_path):
        return None
    
    # SaveData로 시작하는 CSV 파일 찾기
    csv_files = [f for f in os.listdir(restore_path) 
                 if f.endswith('.csv') and 'SaveData' in f and 'SaveEndData' not in f]
    csv_files.sort()  # 숫자 순서대로 정렬
    
    if not csv_files:
        print(f"⚠️  SaveData 파일을 찾을 수 없습니다: {restore_path}")
        return None
    
    # 최대 파일 개수 제한
    csv_files = csv_files[:max_files]
    
    dataframes = []
    for file in csv_files:
        try:
            file_path = os.path.join(restore_path, file)
            df_temp = pd.read_csv(file_path, sep=',', skiprows=0, engine='c',
                                 header=None, encoding='cp949', on_bad_lines='skip')
            dataframes.append(df_temp)
        except Exception as e:
            print(f"⚠️  파일 로딩 실패 ({file}): {e}")
            continue
    
    if dataframes:
        df_combined = pd.concat(dataframes, ignore_index=True)
        print(f"✅ PNE 프로파일 데이터 로드 완료: {len(csv_files)}개 파일, {len(df_combined)} rows")
        return df_combined
    else:
        return None


# ========== Toyo 데이터 로딩 함수 ==========

def find_toyo_channel_folders(path):
    """
    Toyo 채널 폴더 찾기 (숫자로만 이루어진 폴더)
    
    Parameters:
        path (str): Toyo 데이터 경로
    
    Returns:
        list: 채널 폴더 경로 리스트
    """
    if not os.path.exists(path):
        return []
    
    channel_folders = []
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path) and item.isdigit():
            channel_folders.append(item_path)
    
    channel_folders.sort()
    return channel_folders


def load_toyo_cycle_data(channel_path):
    """
    Toyo 사이클 데이터 로딩 (capacity.log)
    
    Parameters:
        channel_path (str): Toyo 채널 폴더 경로
    
    Returns:
        pd.DataFrame or None: 사이클 데이터 DataFrame
    """
    capacity_file = os.path.join(channel_path, 'capacity.log')
    
    if not os.path.isfile(capacity_file):
        print(f"⚠️  capacity.log 파일이 없습니다: {channel_path}")
        return None
    
    try:
        df = pd.read_csv(capacity_file, sep=',', skiprows=0, engine='c', 
                        encoding='cp949', on_bad_lines='skip')
        
        # 컬럼명 표준화 (Toyo 충방전기 타입에 따라 다를 수 있음)
        if 'Cap[mAh]' in df.columns:
            # BLK 3600/3000 타입
            df = df[['TotlCycle', 'Condition', 'Cap[mAh]', 'Ocv', 'PeakTemp[Deg]', 'AveVolt[V]']]
            df.columns = ['Cycle', 'Condition', 'Capacity_mAh', 'OCV_V', 'Temp_C', 'AvgVolt_V']
        elif 'Capacity[mAh]' in df.columns:
            # BLK 5200 타입
            df = df[['Total Cycle', 'Condition', 'Capacity[mAh]', 'OCV[V]', 'Peak Temp.[deg]', 'Ave. Volt.[V]']]
            df.columns = ['Cycle', 'Condition', 'Capacity_mAh', 'OCV_V', 'Temp_C', 'AvgVolt_V']
        
        print(f"✅ Toyo 사이클 데이터 로드 완료: {len(df)} rows")
        return df
        
    except Exception as e:
        print(f"❌ Toyo 사이클 데이터 로딩 실패: {e}")
        return None


def load_toyo_profile_data(channel_path, max_cycles=5):
    """
    Toyo 프로파일 데이터 로딩 (%06d 형식 파일)
    
    Parameters:
        channel_path (str): Toyo 채널 폴더 경로
        max_cycles (int): 로드할 최대 사이클 개수
    
    Returns:
        pd.DataFrame or None: 프로파일 데이터 DataFrame
    """
    dataframes = []
    
    for cycle in range(1, max_cycles + 1):
        profile_file = os.path.join(channel_path, f"{cycle:06d}")
        
        if not os.path.isfile(profile_file):
            continue
        
        try:
            df_temp = pd.read_csv(profile_file, sep=',', skiprows=3, engine='c',
                                 encoding='cp949', on_bad_lines='skip')
            
            # 컬럼명 표준화
            if 'PassTime[Sec]' in df_temp.columns:
                df_temp = df_temp[['PassTime[Sec]', 'Voltage[V]', 'Current[mA]', 'Condition']]
                df_temp.columns = ['Time_s', 'Voltage_V', 'Current_mA', 'Condition']
            elif 'Passed Time[Sec]' in df_temp.columns:
                df_temp = df_temp[['Passed Time[Sec]', 'Voltage[V]', 'Current[mA]', 'Condition']]
                df_temp.columns = ['Time_s', 'Voltage_V', 'Current_mA', 'Condition']
            
            df_temp['Cycle'] = cycle
            dataframes.append(df_temp)
            
        except Exception as e:
            print(f"⚠️  사이클 {cycle} 파일 로딩 실패: {e}")
            continue
    
    if dataframes:
        df_combined = pd.concat(dataframes, ignore_index=True)
        print(f"✅ Toyo 프로파일 데이터 로드 완료: {len(dataframes)}개 사이클, {len(df_combined)} rows")
        return df_combined
    else:
        print(f"⚠️  프로파일 데이터를 찾을 수 없습니다: {channel_path}")
        return None


print("✅ 데이터 로딩 함수 정의 완료")

## 6. 원시 데이터 로딩 실행

각 경로의 사이클러 타입에 따라 적절한 데이터 로딩

In [ ]:
# 로드된 데이터를 저장할 딕셔너리
loaded_data = {
    'pne_cycle': {},
    'pne_profile': {},
    'toyo_cycle': {},
    'toyo_profile': {}
}

# 각 경로별로 데이터 로딩
for idx, row in df_results.iterrows():
    path = row['path']
    folder_name = row['folder_name']
    cycler_type = row['cycler_type']
    
    print(f"\n{'='*60}")
    print(f"📁 경로: {folder_name}")
    print(f"🔧 타입: {cycler_type}")
    print(f"{'='*60}")
    
    if cycler_type == 'PNE':
        # PNE 사이클 데이터 로딩
        cycle_df = load_pne_cycle_data(path)
        if cycle_df is not None:
            loaded_data['pne_cycle'][folder_name] = cycle_df
        
        # PNE 프로파일 데이터 로딩 (처음 5개 파일만)
        profile_df = load_pne_profile_data(path, max_files=5)
        if profile_df is not None:
            loaded_data['pne_profile'][folder_name] = profile_df
    
    elif cycler_type == 'Toyo':
        # Toyo 채널 폴더 찾기
        channel_folders = find_toyo_channel_folders(path)
        
        if channel_folders:
            print(f"  📂 발견된 채널 폴더: {len(channel_folders)}개")
            
            # 첫 번째 채널만 로드 (예시)
            first_channel = channel_folders[0]
            channel_name = os.path.basename(first_channel)
            
            print(f"  🔄 채널 {channel_name} 로딩 중...")
            
            # Toyo 사이클 데이터 로딩
            cycle_df = load_toyo_cycle_data(first_channel)
            if cycle_df is not None:
                loaded_data['toyo_cycle'][f"{folder_name}_ch{channel_name}"] = cycle_df
            
            # Toyo 프로파일 데이터 로딩 (처음 3개 사이클만)
            profile_df = load_toyo_profile_data(first_channel, max_cycles=3)
            if profile_df is not None:
                loaded_data['toyo_profile'][f"{folder_name}_ch{channel_name}"] = profile_df
        else:
            print(f"  ⚠️  채널 폴더를 찾을 수 없습니다")

print(f"\n\n{'='*60}")
print("📊 데이터 로딩 요약")
print(f"{'='*60}")
print(f"  PNE 사이클 데이터: {len(loaded_data['pne_cycle'])}개")
print(f"  PNE 프로파일 데이터: {len(loaded_data['pne_profile'])}개")
print(f"  Toyo 사이클 데이터: {len(loaded_data['toyo_cycle'])}개")
print(f"  Toyo 프로파일 데이터: {len(loaded_data['toyo_profile'])}개")
print(f"{'='*60}")

## 7. 로드된 데이터 확인

In [ ]:
# PNE 사이클 데이터 샘플
if loaded_data['pne_cycle']:
    first_key = list(loaded_data['pne_cycle'].keys())[0]
    print(f"📌 PNE 사이클 데이터 샘플 ({first_key}):")
    display(loaded_data['pne_cycle'][first_key].head(10))
else:
    print("PNE 사이클 데이터가 없습니다.")

print("\n" + "="*60 + "\n")

# Toyo 사이클 데이터 샘플
if loaded_data['toyo_cycle']:
    first_key = list(loaded_data['toyo_cycle'].keys())[0]
    print(f"📌 Toyo 사이클 데이터 샘플 ({first_key}):")
    display(loaded_data['toyo_cycle'][first_key].head(10))
else:
    print("Toyo 사이클 데이터가 없습니다.")

## 8. 결과 확인

In [ ]:
# 전체 결과 DataFrame 출력
df_results